In [23]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
df=pd.read_csv("kiwilytics_orders.csv")

In [3]:
df.shape

(100, 6)

In [12]:
df.head()

,order_id,customer_name,product,quantity,unit_price,order_date
0,1,Maria,Kiwi Chips,2,3.0,2024-01-29
1,2,Richard,Kiwi Chips,1,3.0,2024-01-08
2,3,Nicholas,Kiwi Candy,1,2.5,2024-01-25
3,4,Raymond,Kiwi Candy,4,NaN,2024-01-04
4,5,David,Kiwi Juice,1,4.5,2024-02-25


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_id       100 non-null    int64  
 1   customer_name  100 non-null    object 
 2   product        100 non-null    object 
 3   quantity       100 non-null    int64  
 4   unit_price     92 non-null     float64
 5   order_date     100 non-null    object 
dtypes: float64(1), int64(2), object(3)
memory usage: 4.8+ KB


In [9]:
df.isna().sum()

order_id         0
customer_name    0
product          0
quantity         0
unit_price       8
order_date       0
dtype: int64

In [11]:

df.describe()

,order_id,quantity,unit_price
count,100.000000,100.000000,92.000000
mean,50.500000,2.910000,3.961957
std,29.011492,1.371389,1.326471
min,1.000000,1.000000,2.500000
25%,25.750000,2.000000,2.500000
50%,50.500000,3.000000,3.750000
75%,75.250000,4.000000,5.500000
max,100.000000,5.000000,6.000000


In [ ]:

df["order_date"]=pd.to_datetime(df["order_date"],errors="coerce")

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_id       100 non-null    int64         
 1   customer_name  100 non-null    object        
 2   product        100 non-null    object        
 3   quantity       100 non-null    int64         
 4   unit_price     92 non-null     float64       
 5   order_date     100 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 4.8+ KB


In [ ]:

df.groupby("product")["unit_price"].mean()

product
Kiwi Candy       2.5
Kiwi Chips       3.0
Kiwi Jam         6.0
Kiwi Juice       4.5
Kiwi Smoothie    5.5
Name: unit_price, dtype: float64

In [16]:
avg_price_per_product = (
    df.groupby("product")["unit_price"]
      .mean()
)
avg_price_per_product

product
Kiwi Candy       2.5
Kiwi Chips       3.0
Kiwi Jam         6.0
Kiwi Juice       4.5
Kiwi Smoothie    5.5
Name: unit_price, dtype: float64

In [17]:
df["final_unit_price"] = df["unit_price"].fillna(
    df["product"].map(avg_price_per_product)
)

In [18]:
df.head()

,order_id,customer_name,product,quantity,unit_price,order_date,final_unit_price
0,1,Maria,Kiwi Chips,2,3.0,2024-01-29,3.0
1,2,Richard,Kiwi Chips,1,3.0,2024-01-08,3.0
2,3,Nicholas,Kiwi Candy,1,2.5,2024-01-25,2.5
3,4,Raymond,Kiwi Candy,4,NaN,2024-01-04,2.5
4,5,David,Kiwi Juice,1,4.5,2024-02-25,4.5


In [19]:
df["final_unit_price"].sum()
df["revenue"]= df["quantity"] * df["final_unit_price"]
df.head()

,order_id,customer_name,product,quantity,unit_price,order_date,final_unit_price,revenue
0,1,Maria,Kiwi Chips,2,3.0,2024-01-29,3.0,6.0
1,2,Richard,Kiwi Chips,1,3.0,2024-01-08,3.0,3.0
2,3,Nicholas,Kiwi Candy,1,2.5,2024-01-25,2.5,2.5
3,4,Raymond,Kiwi Candy,4,NaN,2024-01-04,2.5,10.0
4,5,David,Kiwi Juice,1,4.5,2024-02-25,4.5,4.5


In [31]:
df_revenue_distribution=df.groupby("order_date")["revenue"].sum().reset_index()
px.line(
    df_revenue_distribution,
    x="order_date",
    y="revenue",
    title="Daily Revenue Over Time",
    labels={"order_date":"Order Date","revenue":"Total Revenue"}
)

In [20]:
df["revenue"].sum()

np.float64(1167.5)

In [29]:
df_top_products=df.groupby("product")["quantity"].sum().sort_values(ascending=False)

In [30]:
px.bar(
    df_top_products.head(10),
    title="Top 10 Products by Quantity Sold",
    labels={"index":"Product","value":"Total Quantity Sold"}
    )

In [26]:
df_top_customers=df.groupby("customer_name")["revenue"].sum().sort_values(ascending=False)
df_top_customers

customer_name
Eric        100.5
Ashley       64.0
James        37.0
Devin        30.0
Latasha      30.0
            ...  
Kristin       4.5
Justin        4.5
Alex          2.5
Jesse         2.5
Nicholas      2.5
Name: revenue, Length: 75, dtype: float64

In [28]:
px.bar(
    df_top_customers.head(10),
    title="Top 10 Customers by Revenue",
    labels={"index":"Customer Name","value":"Total Revenue"}
    )